# Reddit Video Processor for YouTube - Google Colab

This notebook allows you to run your Reddit video processing pipeline in the Google Colab environment. It handles all the necessary setup, dependency installation, and file operations to process and upload videos from Reddit to YouTube.

## 1. Setup Environment

First, we'll install all the necessary dependencies required by your video processing script.

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/beenycool/yotuubef
%cd yotuubef

# Install necessary packages
!apt-get update
!apt-get install -y ffmpeg imagemagick

# Install Python packages
!pip install -q yt-dlp moviepy praw numpy pillow google-api-python-client google-auth-oauthlib google-auth-httplib2 psutil google-generativeai elevenlabs joblib opencv-python scikit-learn pyyaml

### Configure ImageMagick Policy

MoviePy requires ImageMagick for text generation, so we need to modify the ImageMagick policy to allow these operations in the Colab environment.

In [ ]:
# Configure ImageMagick to allow text operations
!sed -i 's/rights="none" pattern="PS"/rights="read|write" pattern="PS"/' /etc/ImageMagick-6/policy.xml
!sed -i 's/rights="none" pattern="PDF"/rights="read|write" pattern="PDF"/' /etc/ImageMagick-6/policy.xml
!sed -i 's/rights="none" pattern="XPS"/rights="read|write" pattern="XPS"/' /etc/ImageMagick-6/policy.xml

print("ImageMagick policy configured for text operations.")

## 2. Upload Project Files

Upload your project files from your local machine to Google Colab.

In [ ]:
from google.colab import files
import os
import shutil
import pathlib

# Create project directories
os.makedirs("music", exist_ok=True)
os.makedirs("temp_processing", exist_ok=True)

# Verify required files exist in cloned repo
required_files = ['script.py', 'video_processor.py', 'config.yaml']
missing_files = []

for file in required_files:
    if not os.path.exists(file):
        missing_files.append(file)

if missing_files:
    raise FileNotFoundError(f"Missing required files in repository: {', '.join(missing_files)}")

print("All required files found in repository")

### Upload Music Files

Upload the music files for your video processing pipeline.

In [ ]:
print("Please upload the music files from your local 'music' folder:")
uploaded_music = files.upload()

# Move music files to the music directory
for filename in uploaded_music.keys():
    shutil.move(filename, os.path.join("music", filename))
    print(f"Moved {filename} to music folder")

# List the music files
print("\nMusic files available:")
!ls -la music/

### Upload Watermark (Optional)

If you use a watermark in your videos, upload it here.

In [ ]:
print("Please upload the watermark.png file (if you have one):")
uploaded_watermark = files.upload()

for filename in uploaded_watermark.keys():
    print(f"Uploaded: {filename}")

### Upload Google Client Secrets

Upload your Google client secrets JSON file for YouTube API authentication.

In [ ]:
# Google Client Secrets (One-time setup)
print("For YouTube uploads, you need to upload your client_secrets.json file.")
print("This only needs to be done once per Colab session.")
print("The file will be saved in Colab's temporary storage.")

if not os.path.exists("client_secrets.json"):
    print("\nPlease upload your client_secrets.json file:")
    uploaded_secrets = files.upload()
    
    client_secrets_file = ""
    for filename in uploaded_secrets.keys():
        if "client_secret" in filename and filename.endswith(".json"):
            client_secrets_file = filename
            print(f"Saved {filename} for this session")
            break
    
    if not client_secrets_file:
        print("WARNING: No valid client_secrets.json uploaded. YouTube uploads disabled.")
else:
    print("\nUsing existing client_secrets.json from previous upload")
    client_secrets_file = "client_secrets.json"

## 3. Configure API Keys & Environment Variables

Set up the necessary API keys and environment variables for the script to run.

In [ ]:
import os

# Reddit API Credentials
reddit_client_id = input("Enter your Reddit Client ID: ")
reddit_client_secret = input("Enter your Reddit Client Secret: ")
reddit_user_agent = input("Enter your Reddit User Agent (or press Enter for default): ") or "python:VideoBot:v1.5 (by /u/YOUR_USERNAME)"

# Gemini API Key
gemini_api_key = input("Enter your Google Gemini API Key: ")

# ElevenLabs API Key (optional)
elevenlabs_api_key = input("Enter your ElevenLabs API Key (or press Enter to skip): ")

# Set environment variables
os.environ["REDDIT_CLIENT_ID"] = reddit_client_id
os.environ["REDDIT_CLIENT_SECRET"] = reddit_client_secret
os.environ["REDDIT_USER_AGENT"] = reddit_user_agent
os.environ["GEMINI_API_KEY"] = gemini_api_key
os.environ["GOOGLE_CLIENT_SECRETS_FILE"] = client_secrets_file if client_secrets_file else ""

if elevenlabs_api_key:
    os.environ["ELEVENLABS_API_KEY"] = elevenlabs_api_key

print("\nEnvironment variables set successfully!")

## 4. Run the Script

Now we'll run the script with subreddits from config.yaml

In [ ]:
import yaml

try:
    # Load config.yaml
    with open('config.yaml') as f:
        config = yaml.safe_load(f)
    
    # Get subreddits from config
    subreddits = config.get('subreddits', [])
    if not subreddits:
        raise ValueError("No subreddits found in config.yaml")
    
    # Set default options
    max_videos = "3"
    skip_upload_flag = "--skip_upload"
    
    # Build command with all subreddits
    subreddits_str = " ".join(subreddits)
    command = f"python script.py {subreddits_str} --max_videos {max_videos} {skip_upload_flag}"
    
    print(f"\nProcessing videos from subreddits: {', '.join(subreddits)}")
    print(f"Executing: {command}")
    
    !{command}
    
except Exception as e:
    print(f"Error loading configuration: {e}")
    print("Please ensure config.yaml exists and contains valid subreddit list")

## 5. Download Processed Videos

If you've created videos and want to download them to your local machine, you can do that here.

In [ ]:
import glob
from google.colab import files

# Find output videos
output_videos = glob.glob("temp_processing/*_final.mp4")

if output_videos:
    print(f"Found {len(output_videos)} processed videos:")
    for video in output_videos:
        print(f"- {os.path.basename(video)}")
    
    download_choice = input("\nDo you want to download these videos? (y/n): ").lower()
    if download_choice == "y":
        for video in output_videos:
            try:
                print(f"Downloading {os.path.basename(video)}...")
                files.download(video)
            except Exception as e:
                print(f"Error downloading {video}: {e}")
else:
    print("No processed videos found in the temp_processing directory.")

## 6. Clean Up

Optionally clean up temporary files to free up space.

In [ ]:
import shutil

cleanup = input("Do you want to clean up temporary files? (y/n): ").lower()
if cleanup == "y":
    try:
        files_removed = 0
        if os.path.exists("temp_processing"):
            for file in glob.glob("temp_processing/*"):
                if not file.endswith("_final.mp4"):  # Keep final videos
                    os.remove(file)
                    files_removed += 1
        print(f"Removed {files_removed} temporary files.")
    except Exception as e:
        print(f"Error during cleanup: {e}")

## Troubleshooting Guide

If you encounter issues running the script in Colab, here are some common solutions:

1. **Memory Issues**: If you get out-of-memory errors, try:

   - Restart the runtime and rerun with fewer videos
   - Upgrade to Colab Pro for more memory

2. **GPU Acceleration**: To enable GPU acceleration:

   - Go to Runtime → Change runtime type → Select GPU as Hardware accelerator

3. **Authentication Issues**: For YouTube API authentication:

   - Make sure your client_secrets.json is properly uploaded
   - Follow the authentication link that appears in the output

4. **Missing Dependencies**: If you get module not found errors:

   - Rerun the dependency installation cell
   - Check for specific error messages and install the missing package

5. **File Permissions**: If you get permission errors:

   - Ensure the ImageMagick policy configuration was successful
   - Try running with simpler parameters (disable some features)